In [1]:
# Laura Chapman
# Script for developing statistical tools for Glue
# Computes statistics for subsets as well as entire data using compute_statistic
# Changed from astropy tables to pandas dataframe

# Format data well in a popup using qt
# Color code by subset

In [2]:
# Basic code that imports glue and loads in and links the data

import sys
from glue.core.data_factories import load_data
from glue.core import DataCollection
from glue.core.link_helpers import LinkSame
from glue.app.qt.application import GlueApplication
from glue.viewers.image.qt import ImageViewer
from glue_vispy_viewers.volume.volume_viewer import VispyVolumeViewer

image_filename='w5.fits'
catalog_filename='w5_psc.vot'

#load 2 datasets from files
catalog = load_data(catalog_filename)
image = load_data(image_filename)

dc = DataCollection([catalog,image])

# link positional information
dc.add_link(LinkSame(catalog.id['RAJ2000'], image.id['Right Ascension']))
dc.add_link(LinkSame(catalog.id['DEJ2000'], image.id['Declination']))

#Create subset based on filament mask
ra_state=(image.id['Right Ascension'] > 44) & (image.id['Right Ascension'] < 46)
subset_group=dc.new_subset_group('RA_Selection',ra_state)
subset_group.style.color = '#0000FF'

#start Glue
# app = GlueApplication(dc)

# imageviewer = app.new_data_viewer(ImageViewer)
# imageviewer.add_data(image)

# app.start()



In [3]:
dc.data[0].label
dc.data[0].components

[Pixel Axis 0 [x],
 World 0,
 ID,
 RAJ2000,
 DEJ2000,
 Jmag,
 Hmag,
 Ksmag,
 __3.6_,
 __4.5_,
 __5.8_,
 __8.0_,
 __24_,
 Type,
 __4.5__-__5.8_,
 __5.8__-__8.0_]

In [4]:
import glue.utils.array as gua
import glue.core.data as gcd
from astropy.table import Table

In [5]:
# Constructs a pandas DataFrame instead of an astropy table

import pandas as pd
from pandas import DataFrame
import numpy as np

In [6]:
# Defines a subset and runs statistics using compute_statistic for the subset
# Saves data using a pandas dataframe called my_pandas_data

# Data that subset is pulled from
data = dc[0]

# Define state and subset
state1 = data.id['Jmag'] > 14
subset1 = data.new_subset(state1, label='Jmag > 14')

# Same arrays as with full data
mean_array = []
median_array = []
min_array = []
max_array = []
sum_array = []
name_array = []
tables = []

headings = ('mean', 'median', 'minimum', 'maximum', 'sum')

print(data.label, 'subset1')
print() 
for j in range (0, len(data.components)):
    name = data.components[j].label # Get the name of each component
    name_array.append(name) # add to the name array to build the table
    mean_array.append(data.compute_statistic('mean', subset1.components[j], subset_state=subset1.subset_state))
    median_array.append(data.compute_statistic('median', subset1.components[j], subset_state=subset1.subset_state))       
    min_array.append(data.compute_statistic('minimum', subset1.components[j], subset_state=subset1.subset_state))       
    max_array.append(data.compute_statistic('maximum', subset1.components[j], subset_state=subset1.subset_state))      
    sum_array.append(data.compute_statistic('sum', subset1.components[j], subset_state=subset1.subset_state))        
   
column_data = np.asarray([mean_array, median_array, min_array, max_array, sum_array]).transpose()

my_pandas_data = pd.DataFrame(column_data, index=name_array, columns=headings)

w5_psc subset1



In [7]:
type(dc[0])

glue.core.data.Data

In [8]:
array1 = [1,3,5,6]
array2 = [4,5,6,6]
array3 = [6,7,8,9]

colnames = ['array1', 'array2', 'array3']
rownames = ['col1', 'col2', 'col3', 'col4']

data = np.asarray([array1, array2, array3]).transpose()

pandas_data = pd.DataFrame(data, index=rownames, columns=colnames)

In [117]:
# Full working example! 
# Has a dropdown menu with the components- highlights the row of the component selected
# Still need to figure out how to resize window to fit the contents
# Merged into one class and takes in a data object as a parameter

import sys
from PyQt5 import QtCore, QtWidgets
Qt = QtCore.Qt
from PyQt5.QtCore import QVariant
from qtpy.QtWidgets import (QApplication, QLabel, QWidget, QComboBox,
                            QVBoxLayout, QPushButton, QGridLayout, QTableView) 

class stat_widget(QWidget):
    def __init__(self, data_collection, parent = None):
        super(stat_widget, self).__init__(parent)
        self.view = QtCore.QAbstractTableModel.__init__(self, parent)
        self.dc = data_collection  
        
        self.mean_array = []
        self.median_array = []
        self.min_array = []
        self.max_array = []
        self.sum_array = []
        self.name_array = []
        
        self.layout = QVBoxLayout()
        self.cb_data = QComboBox()
        
        for i in range(0, len(self.dc.data)):
            self.cb_data.addItem(self.dc.data[i].label)
        
        self.cb_data.currentIndexChanged.connect(self.dataSelectionChange)
    
        self.layout.addWidget(self.cb_data)  
        self.cb_components = QComboBox()
        # Ideally make this a default fill box
        self.cb_components.addItem('Choose a component')  
        self.cb_components.currentIndexChanged.connect(self.runStats)  
        
        self.layout.addWidget(self.cb_components)
        
#         self.add_row = QPushButton('Add a row')
#         self.add_row.clicked.connect(self.addRow)
#         self.layout.addWidget(self.add_row, 1,2)
        self.view = QTableView()
        self.layout.addWidget(self.view)

        self.setLayout(self.layout)
        self.setWindowTitle("Statistics")          
     
    def runStats(self, i):
        comp_i = i
        print('running stats for ', self.dc[self.data_index].components[comp_i].label)
        self.mean_array = []
        self.median_array = []
        self.min_array = []
        self.max_array = []
        self.sum_array = []
        self.name_array = []
        headings = ('mean', 'median', 'minimum', 'maximum', 'sum')

#         for j in range(0, len(self.dc[self.data_index].components[i])):
#             name = self.core.components[j].label # Get the name of each component
#             name_array.append(name) # add to the name array to build the table
#             mean_array.append(self.core.compute_statistic('mean', self.core.components[j]))
#             median_array.append(self.core.compute_statistic('median', self.core.components[j]))       
#             min_array.append(self.core.compute_statistic('minimum', self.core.components[j]))       
#             max_array.append(self.core.compute_statistic('maximum', self.core.components[j]))      
#             sum_array.append(self.core.compute_statistic('sum', self.core.components[j]))         
        
            # At this point we have several column arrays to put into a pandas data frame
        name = dc[self.data_index].label + '--' + self.dc[self.data_index].components[comp_i].label
        self.name_array.append(name) # add to the name array to build the table
        self.mean_array.append(self.dc[self.data_index].compute_statistic('mean', self.dc[self.data_index].components[comp_i]))
        self.median_array.append(self.dc[self.data_index].compute_statistic('median', self.dc[self.data_index].components[comp_i]))       
        self.min_array.append(self.dc[self.data_index].compute_statistic('minimum', self.dc[self.data_index].components[comp_i]))      
        self.max_array.append(self.dc[self.data_index].compute_statistic('maximum', self.dc[self.data_index].components[comp_i]))     
        self.sum_array.append(self.dc[self.data_index].compute_statistic('sum', self.dc[self.data_index].components[comp_i]))            
                       
        column_data = np.asarray([self.mean_array, self.median_array, self.min_array, self.max_array, self.sum_array]).transpose()

        self.data_frame = pd.DataFrame(column_data, index=self.name_array, columns=headings)             
        model = pandasModel(self.data_frame)
        
#         self.layout.removeWidget(self.view)

        self.view.setModel(model)
        self.view.setShowGrid(False)

        self.layout.addWidget(self.view)                                              
        
    def dataSelectionChange(self,i):
        self.data_index = i  
        self.cb_components.clear()
        for j in range (0, len(self.dc[self.data_index].components)):
            self.cb_components.addItem(self.dc[self.data_index].components[j].label)
                

#     def addRow(self):
#         self.count_adds+=1
#         self.cb_addData = QComboBox()
#         for i in range(0, len(self.dc.data)):
#             self.cb_addData.addItem(self.dc.data[i].label)
#         self.cb_addData.currentIndexChanged.connect(self.addDataSelectionChange) 
#         self.layout.addWidget(self.cb_addData, self.count_adds, 0)   
    
#         self.cb_addComponents = QComboBox()
#         self.cb_addComponents.addItem('Choose a component')       
        
#         self.layout.addWidget(self.cb_addComponents, self.count_adds, 1)    
    
#     def addDataSelectionChange(self,i):
#         self.data_index = i  
#         self.cb_addComponents.clear()
#         for j in range (0, len(self.dc[self.data_index].components)):
#             self.cb_addComponents.addItem(self.dc[self.data_index].components[j].label)
#         self.cb_addComponents.currentIndexChanged.connect(self.runStats)     
    
class pandasModel(QtCore.QAbstractTableModel):
    def __init__(self, df, parent=None):
        QtCore.QAbstractTableModel.__init__(self, parent)
        self.data_frame = df
        super(pandasModel, self).__init__(parent)      

    def rowCount(self, parent=None):
        return len(self.data_frame.values)

    def columnCount(self, parent=None):
        return self.data_frame.columns.size

    def data(self, index, role=Qt.DisplayRole):
        if index.isValid():
            if role == Qt.DisplayRole:
                return QVariant(str(
                    self.data_frame.values[index.row()][index.column()]))
        return QVariant()
    
    def headerData(self, col, orientation, role):
        if orientation == Qt.Horizontal and role == Qt.DisplayRole:
            return QVariant(self.data_frame.columns[col])
        if orientation == Qt.Vertical and role == Qt.DisplayRole:
            return QVariant(self.data_frame.index[col])
        return QVariant()                   
            
app = QApplication.instance()
if app is None:
    app = QApplication(sys.argv)
else:
    print('QApplication instance already exists: %s' % str(app))
    
ex = stat_widget(dc)
ex.show()
sys.exit(app.exec_())

QApplication instance already exists: <PyQt5.QtWidgets.QApplication object at 0x1511aaf798>
running stats for  PRIMARY
running stats for  Pixel Axis 0 [y]
running stats for  Declination
running stats for  Right Ascension
running stats for  __5.8__-__8.0_
running stats for  Pixel Axis 0 [x]
running stats for  Jmag


SystemExit: 0

/Users/laurachapman/anaconda/envs/glueviz-dev/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
